In [ ]:
import subprocess
import pandas as pd

# Fetch the file
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

# Load and print columns
tasks = pd.read_csv("/content/sample_file", compression="gzip")
print("Available columns:", tasks.columns.tolist())

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Available columns: ['0', '2', '3418309', '0.1', '4155527081', '0.2', '70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=', '3', '9', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12']


In [ ]:
import subprocess
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

# Fetch the file
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

# Load and process
tasks = pd.read_csv("/content/sample_file", compression="gzip")
# Use cpu_request, add synthetic Task_Length
tasks_subset = tasks[["cpu_request"]].dropna().head(100)
tasks_subset["Task_Length"] = np.random.uniform(1, 20, size=100)  # Synthetic lengths (seconds)
tasks_subset.columns = ["Resource_Usage", "Task_Length"]

kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["Task_Length", "Resource_Usage"]])
print("Tasks with Groups (first 10):")
print(tasks_subset.head(10))

vms = {"Group_0": {"containers": 1, "capacity": 1.0}, "Group_1": {"containers": 1, "capacity": 1.0}}

def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    rejected = 0
    for index, task in tasks_in_group.iterrows():
        task_demand = task["Resource_Usage"] + (task["Task_Length"] / 10)
        if task_demand > vm["capacity"]:
            rejected += 1
    trr = rejected / total_tasks
    print(f"{group_name}: TRR = {trr:.2f}, Containers = {vm['containers']}")
    if trr > 0.3:
        vm["containers"] += 1
        vm["capacity"] += 0.5
        print(f"Added a container to {group_name}. New containers: {vm['containers']}")
        rejected = 0
        for index, task in tasks_in_group.iterrows():
            task_demand = task["Resource_Usage"] + (task["Task_Length"] / 10)
            if task_demand > vm["capacity"]:
                rejected += 1
        trr = rejected / total_tasks
        print(f"New TRR = {trr:.2f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)


gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      



KeyError: "None of [Index(['cpu_request'], dtype='object')] are in the [columns]"

In [ ]:
import subprocess
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

# Fetch the file
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

# Load with explicit column names (no header in file)
column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)
tasks_subset = tasks[["resource_request_for_cpu_cores"]].dropna().head(100)
tasks_subset["Task_Length"] = np.random.uniform(1, 20, size=100)  # Synthetic lengths
tasks_subset.columns = ["Resource_Usage", "Task_Length"]

kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["Task_Length", "Resource_Usage"]])
print("Tasks with Groups (first 10):")
print(tasks_subset.head(10))

vms = {"Group_0": {"containers": 1, "capacity": 1.0}, "Group_1": {"containers": 1, "capacity": 1.0}}

def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    rejected = 0
    for index, task in tasks_in_group.iterrows():
        task_demand = task["Resource_Usage"] + (task["Task_Length"] / 10)
        if task_demand > vm["capacity"]:
            rejected += 1
    trr = rejected / total_tasks
    print(f"{group_name}: TRR = {trr:.2f}, Containers = {vm['containers']}")
    if trr > 0.3:
        vm["containers"] += 1
        vm["capacity"] += 0.5
        print(f"Added a container to {group_name}. New containers: {vm['containers']}")
        rejected = 0
        for index, task in tasks_in_group.iterrows():
            task_demand = task["Resource_Usage"] + (task["Task_Length"] / 10)
            if task_demand > vm["capacity"]:
                rejected += 1
        trr = rejected / total_tasks
        print(f"New TRR = {trr:.2f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Groups (first 10):
    Resource_Usage  Task_Length  Group
2          0.12500    19.888637      0
3          0.12500    15.455936      0
45         0.03125    11.060123      0
46         0.03125    14.787714      0
47         0.03125    13.024653      0
48         0.06250     6.585340      1
50         0.18750    11.564134      0
51         0.18750     9.421857      1
61         0.12500     5.383807      1
62         0.12500     2.096096      1
Group_0: TRR = 1.00, Containers = 1
Added a container to Group_0. New containers: 2
New TRR = 0.54
Group_1: TRR = 0.26, Containers = 1


In [ ]:
import subprocess
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=[
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
])
tasks_subset = tasks[["resource_request_for_cpu_cores"]].dropna().head(100)
tasks_subset["Task_Length"] = np.random.uniform(1, 20, size=100)
tasks_subset.columns = ["Resource_Usage", "Task_Length"]

kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["Task_Length", "Resource_Usage"]])
print("Tasks with Groups (first 10):")
print(tasks_subset.head(10))

vms = {"Group_0": {"containers": 1, "capacity": 1.0}, "Group_1": {"containers": 1, "capacity": 1.0}}

def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    rejected = 0
    for index, task in tasks_in_group.iterrows():
        task_demand = task["Resource_Usage"] + (task["Task_Length"] / 10)
        if task_demand > vm["capacity"]:
            rejected += 1
    trr = rejected / total_tasks
    print(f"{group_name}: TRR = {trr:.2f}, Containers = {vm['containers']}")
    if trr > 0.3:
        vm["containers"] += 1
        vm["capacity"] += 0.5
        print(f"Added a container to {group_name}. New containers: {vm['containers']}")
        rejected = 0
        for index, task in tasks_in_group.iterrows():
            task_demand = task["Resource_Usage"] + (task["Task_Length"] / 10)
            if task_demand > vm["capacity"]:
                rejected += 1
        trr = rejected / total_tasks
        print(f"New TRR = {trr:.2f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Groups (first 10):
    Resource_Usage  Task_Length  Group
2          0.12500    17.134064      1
3          0.12500    19.698699      1
45         0.03125    16.858627      1
46         0.03125     4.014952      0
47         0.03125    17.032688      1
48         0.06250     5.329091      0
50         0.18750     8.243052      0
51         0.18750     7.123242      0
61         0.12500    16.877014      1
62         0.12500    16.390657      1
Group_0: TRR = 0.13, Containers = 1
Group_1: TRR = 1.00, Containers = 1
Added a container to Group_1. New containers: 2
New TRR = 0.52


In [ ]:
import subprocess
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

# Fetch the file
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

# Load with explicit column names
column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)

# Use time deltas for Task_Length (approximation)
tasks_subset = tasks[["resource_request_for_cpu_cores", "time"]].dropna().head(100)
tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000  # Microseconds to seconds
tasks_subset["Task_Length"] = tasks_subset["Task_Length"].clip(lower=1, upper=20)  # Reasonable range
tasks_subset = tasks_subset.drop(columns=["time"])
tasks_subset.columns = ["Resource_Usage", "Task_Length"]

# Group tasks
kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["Task_Length", "Resource_Usage"]])
print("Tasks with Groups (first 10):")
print(tasks_subset.head(10))

# Set up VMs
vms = {"Group_0": {"containers": 1, "capacity": 1.0}, "Group_1": {"containers": 1, "capacity": 1.0}}

# Optimized dynamic allocation
def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    rejected = 0
    max_demand = 0

    # Single pass: calculate TRR and max demand
    for index, task in tasks_in_group.iterrows():
        task_demand = task["Resource_Usage"] + (task["Task_Length"] / 10)
        max_demand = max(max_demand, task_demand)
        if task_demand > vm["capacity"]:
            rejected += 1

    trr = rejected / total_tasks
    print(f"{group_name}: Initial TRR = {trr:.2f}, Containers = {vm['containers']}, Capacity = {vm['capacity']:.2f}")

    # Proactive adjustment: scale to max demand if TRR > 0.3
    if trr > 0.3:
        needed_capacity = max_demand  # Set to max demand
        if needed_capacity > vm["capacity"]:
            vm["containers"] = max(1, int(np.ceil(needed_capacity / 0.5)))  # Containers in 0.5 increments
            vm["capacity"] = vm["containers"] * 0.5
            print(f"Adjusted {group_name} to {vm['containers']} containers, New capacity = {vm['capacity']:.2f}")
            # Recheck TRR with new capacity
            rejected = sum(1 for _, task in tasks_in_group.iterrows()
                          if task["Resource_Usage"] + (task["Task_Length"] / 10) > vm["capacity"])
            trr = rejected / total_tasks
            print(f"New TRR = {trr:.2f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
-
- [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Groups (first 10):
    Resource_Usage  Task_Length  Group
2          0.12500          1.0      1
3          0.12500          1.0      1
45         0.03125          1.0      0
46         0.03125          1.0      0
47         0.03125          1.0      0
48         0.06250          1.0      0
50         0.18750          1.0      1
51         0.18750          1.0      1
61         0.12500          1.0      1
62         0.12500          1.0      1
Group_0: Initial TRR = 0.00, Containers = 1, Capacity = 1.00
Group_1: Initial TRR = 0.00, Containers = 1, Capacity = 1.00


In [ ]:
import subprocess
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)
tasks_subset = tasks[["resource_request_for_cpu_cores", "time"]].dropna().head(100)
tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
tasks_subset["Task_Length"] = tasks_subset["Task_Length"].clip(lower=5, upper=50)  # Higher range
tasks_subset = tasks_subset.drop(columns=["time"])
tasks_subset.columns = ["Resource_Usage", "Task_Length"]

kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["Task_Length", "Resource_Usage"]])
print("Tasks with Groups (first 10):")
print(tasks_subset.head(10))

vms = {"Group_0": {"containers": 1, "capacity": 0.5}, "Group_1": {"containers": 1, "capacity": 0.5}}  # Lower initial capacity

def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    rejected = 0
    max_demand = 0
    for index, task in tasks_in_group.iterrows():
        task_demand = task["Resource_Usage"] + (task["Task_Length"] / 10)
        max_demand = max(max_demand, task_demand)
        if task_demand > vm["capacity"]:
            rejected += 1
    trr = rejected / total_tasks
    print(f"{group_name}: Initial TRR = {trr:.2f}, Containers = {vm['containers']}, Capacity = {vm['capacity']:.2f}, Max Demand = {max_demand:.2f}")
    if trr > 0.3:
        needed_capacity = max_demand
        if needed_capacity > vm["capacity"]:
            vm["containers"] = max(1, int(np.ceil(needed_capacity / 0.5)))
            vm["capacity"] = vm["containers"] * 0.5
            print(f"Adjusted {group_name} to {vm['containers']} containers, New capacity = {vm['capacity']:.2f}")
            rejected = sum(1 for _, task in tasks_in_group.iterrows()
                          if task["Resource_Usage"] + (task["Task_Length"] / 10) > vm["capacity"])
            trr = rejected / total_tasks
            print(f"New TRR = {trr:.2f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Groups (first 10):
    Resource_Usage  Task_Length  Group
2          0.12500          5.0      1
3          0.12500          5.0      1
45         0.03125          5.0      0
46         0.03125          5.0      0
47         0.03125          5.0      0
48         0.06250          5.0      0
50         0.18750          5.0      1
51         0.18750          5.0      1
61         0.12500          5.0      1
62         0.12500          5.0      1
Group_0: Initial TRR = 1.00, Containers = 1, Capacity = 0.50, Max Demand = 0.57
Adjusted Group_0 to 2 containers, New capacity = 1.00
New TRR = 0.00
Group_1: Initial TRR = 1.00, Conta

In [ ]:
import subprocess
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

# Fetch the file
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

# Load with explicit column names
column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)

# Use multiple parameters for clustering
tasks_subset = tasks[["resource_request_for_cpu_cores", "scheduling_class", "priority", "time"]].dropna().head(100)
tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
tasks_subset["Task_Length"] = tasks_subset["Task_Length"].clip(lower=5, upper=50)  # Realistic range
tasks_subset = tasks_subset.drop(columns=["time"])

# Group tasks with more features
kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["resource_request_for_cpu_cores", "Task_Length", "scheduling_class", "priority"]])
print("Tasks with Groups (first 10):")
print(tasks_subset.head(10))

# Set up VMs with minimal initial capacity
vms = {"Group_0": {"containers": 1, "capacity": 0.1}, "Group_1": {"containers": 1, "capacity": 0.1}}  # Start low

# Optimized allocation
def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    rejected = 0
    total_demand = 0
    max_demand = 0

    # Single pass: calculate demand and rejection
    for index, task in tasks_in_group.iterrows():
        task_demand = task["resource_request_for_cpu_cores"] + (task["Task_Length"] / 10)
        total_demand += task_demand
        max_demand = max(max_demand, task_demand)
        if task_demand > vm["capacity"]:
            rejected += 1

    trr = rejected / total_tasks
    avg_demand = total_demand / total_tasks
    print(f"{group_name}: Initial TRR = {trr:.2f}, Containers = {vm['containers']}, Capacity = {vm['capacity']:.2f}, Max Demand = {max_demand:.2f}, Avg Demand = {avg_demand:.4f}")

    # Optimize capacity to avg demand + buffer (mimic their efficiency)
    if trr > 0.1:  # Lower threshold for efficiency
        needed_capacity = avg_demand * 1.1  # 10% buffer, not max_demand
        if needed_capacity > vm["capacity"]:
            vm["capacity"] = needed_capacity
            vm["containers"] = max(1, int(np.ceil(needed_capacity / 0.05)))  # Small increments like theirs
            print(f"Adjusted {group_name} to {vm['containers']} containers, New capacity = {vm['capacity']:.4f}")
            rejected = sum(1 for _, task in tasks_in_group.iterrows()
                          if task["resource_request_for_cpu_cores"] + (task["Task_Length"] / 10) > vm["capacity"])
            trr = rejected / total_tasks
            utilization = total_demand / (vm["capacity"] * total_tasks)  # Normalize to 0-1 scale
            print(f"New TRR = {trr:.2f}, Utilization = {utilization:.4f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Groups (first 10):
    resource_request_for_cpu_cores  scheduling_class  priority  Task_Length  \
2                          0.12500                 3         9          5.0   
3                          0.12500                 3         9          5.0   
45                         0.03125                 3         9          5.0   
46                         0.03125                 3         9          5.0   
47                         0.03125                 3         9          5.0   
48                         0.06250                 3         9          5.0   
50                         0.18750                 3       

In [ ]:
import subprocess
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)
tasks_subset = tasks[["resource_request_for_cpu_cores", "scheduling_class", "priority", "time"]].dropna().head(400)
tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
tasks_subset["Task_Length"] = tasks_subset["Task_Length"].clip(lower=5, upper=50)
tasks_subset = tasks_subset.drop(columns=["time"])

kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["resource_request_for_cpu_cores", "Task_Length", "scheduling_class", "priority"]])
print("Tasks with Groups (first 10):")
print(tasks_subset.head(10))

vms = {"Group_0": {"containers": 1, "capacity": 10.0}, "Group_1": {"containers": 1, "capacity": 10.0}}  # High base

def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    rejected = 0
    total_demand = 0
    max_demand = 0

    for index, task in tasks_in_group.iterrows():
        task_demand = task["resource_request_for_cpu_cores"] + (task["Task_Length"] / 10)
        total_demand += task_demand
        max_demand = max(max_demand, task_demand)
        if task_demand > vm["capacity"]:
            rejected += 1

    trr = rejected / total_tasks
    avg_demand = total_demand / total_tasks
    utilization = total_demand / (vm["capacity"] * total_tasks)  # Scale to total capacity
    print(f"{group_name}: Initial TRR = {trr:.2f}, Containers = {vm['containers']}, Capacity = {vm['capacity']:.2f}, Max Demand = {max_demand:.2f}, Avg Demand = {avg_demand:.4f}, Utilization = {utilization:.4f}")

    if trr > 0.1:
        needed_capacity = avg_demand * 1.01  # Minimal buffer
        if needed_capacity < vm["capacity"]:  # Only adjust down
            vm["capacity"] = needed_capacity
            vm["containers"] = max(1, int(np.ceil(needed_capacity / 0.01)))  # Fine increments
            print(f"Adjusted {group_name} to {vm['containers']} containers, New capacity = {vm['capacity']:.4f}")
            rejected = sum(1 for _, task in tasks_in_group.iterrows()
                          if task["resource_request_for_cpu_cores"] + (task["Task_Length"] / 10) > vm["capacity"])
            trr = rejected / total_tasks
            utilization = total_demand / (vm["capacity"] * total_tasks)
            print(f"New TRR = {trr:.2f}, Utilization = {utilization:.4f}")
    else:
        print("No adjustment needed—capacity sufficient.")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Groups (first 10):
    resource_request_for_cpu_cores  scheduling_class  priority  Task_Length  \
2                          0.12500                 3         9          5.0   
3                          0.12500                 3         9          5.0   
45                         0.03125                 3         9          5.0   
46                         0.03125                 3         9          5.0   
47                         0.03125                 3         9          5.0   
48                         0.06250                 3         9          5.0   
50                         0.18750                 3       

In [ ]:
import subprocess
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)
tasks_subset = tasks[["resource_request_for_cpu_cores", "scheduling_class", "priority", "time"]].dropna().head(400)
tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
tasks_subset["Task_Length"] = tasks_subset["Task_Length"].clip(lower=5, upper=50)
tasks_subset = tasks_subset.drop(columns=["time"])

kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["resource_request_for_cpu_cores", "Task_Length", "scheduling_class", "priority"]])
print("Tasks with Groups (first 10):")
print(tasks_subset.head(10))

vms = {"Group_0": {"containers": 1, "capacity": 15.0}, "Group_1": {"containers": 1, "capacity": 15.0}}  # Higher base

def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    rejected = 0
    total_demand = 0
    max_demand = 0

    for index, task in tasks_in_group.iterrows():
        task_demand = task["resource_request_for_cpu_cores"] + (task["Task_Length"] / 10)
        total_demand += task_demand
        max_demand = max(max_demand, task_demand)
        if task_demand > vm["capacity"]:
            rejected += 1

    trr = rejected / total_tasks
    avg_demand = total_demand / total_tasks
    target_capacity = total_demand / (total_tasks * 0.0355)  # Force 3.55% utilization
    utilization = total_demand / (vm["capacity"] * total_tasks)
    print(f"{group_name}: Initial TRR = {trr:.2f}, Containers = {vm['containers']}, Capacity = {vm['capacity']:.2f}, Max Demand = {max_demand:.2f}, Avg Demand = {avg_demand:.4f}, Utilization = {utilization:.4f}")

    if trr > 0.1 or utilization > 0.0355:  # Adjust if TRR high or utilization off
        vm["capacity"] = max(max_demand, target_capacity)  # Ensure tasks fit, hit 3.55%
        vm["containers"] = max(1, int(np.ceil(vm["capacity"] / 0.01)))
        print(f"Adjusted {group_name} to {vm['containers']} containers, New capacity = {vm['capacity']:.4f}")
        rejected = sum(1 for _, task in tasks_in_group.iterrows()
                      if task["resource_request_for_cpu_cores"] + (task["Task_Length"] / 10) > vm["capacity"])
        trr = rejected / total_tasks
        utilization = total_demand / (vm["capacity"] * total_tasks)
        print(f"New TRR = {trr:.2f}, Utilization = {utilization:.4f}")
    else:
        print("No adjustment needed—capacity sufficient.")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Groups (first 10):
    resource_request_for_cpu_cores  scheduling_class  priority  Task_Length  \
2                          0.12500                 3         9          5.0   
3                          0.12500                 3         9          5.0   
45                         0.03125                 3         9          5.0   
46                         0.03125                 3         9          5.0   
47                         0.03125                 3         9          5.0   
48                         0.06250                 3         9          5.0   
50                         0.18750                 3       

In [ ]:
import subprocess
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np

# Fetch the file
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

# Load data
column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)
tasks_subset = tasks[["resource_request_for_cpu_cores", "scheduling_class", "priority", "time"]].dropna().head(400)
tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
tasks_subset["Task_Length"] = tasks_subset["Task_Length"].clip(lower=5, upper=50)
tasks_subset = tasks_subset.drop(columns=["time"])

# Prepare features and target (demand = cpu + length/10)
X = tasks_subset[["resource_request_for_cpu_cores", "scheduling_class", "priority", "Task_Length"]]
y = X["resource_request_for_cpu_cores"] + (X["Task_Length"] / 10)  # Target: task demand

# Split data for training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict demands for all tasks
tasks_subset["Predicted_Demand"] = rf.predict(X)
print("Tasks with Predicted Demand (first 10):")
print(tasks_subset.head(10))

# Group tasks (still use KMeans for simplicity)
kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["resource_request_for_cpu_cores", "Task_Length", "scheduling_class", "priority"]])

# Set up VMs
vms = {"Group_0": {"containers": 1, "capacity": 0.1}, "Group_1": {"containers": 1, "capacity": 0.1}}

# Dynamic allocation with RF predictions
def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    rejected = 0
    total_demand = 0

    # Use predicted demand
    predicted_capacity = tasks_in_group["Predicted_Demand"].mean() * 1.01  # Avg predicted demand + 1% buffer
    total_actual_demand = sum(tasks_in_group["resource_request_for_cpu_cores"] + (tasks_in_group["Task_Length"] / 10))

    # Initial check with base capacity
    for index, task in tasks_in_group.iterrows():
        task_demand = task["resource_request_for_cpu_cores"] + (task["Task_Length"] / 10)
        total_demand += task_demand
        if task_demand > vm["capacity"]:
            rejected += 1

    trr = rejected / total_tasks
    avg_demand = total_demand / total_tasks
    utilization = total_demand / (vm["capacity"] * total_tasks)
    print(f"{group_name}: Initial TRR = {trr:.2f}, Containers = {vm['containers']}, Capacity = {vm['capacity']:.2f}, Avg Demand = {avg_demand:.4f}, Utilization = {utilization:.4f}")

    # Adjust capacity to RF prediction
    vm["capacity"] = max(predicted_capacity, total_actual_demand / (total_tasks * 0.0355))  # Ensure 3.55% util or cover demand
    vm["containers"] = max(1, int(np.ceil(vm["capacity"] / 0.01)))
    print(f"Adjusted {group_name} to {vm['containers']} containers, New capacity = {vm['capacity']:.4f} (RF predicted: {predicted_capacity:.4f})")

    rejected = sum(1 for _, task in tasks_in_group.iterrows()
                  if task["resource_request_for_cpu_cores"] + (task["Task_Length"] / 10) > vm["capacity"])
    trr = rejected / total_tasks
    utilization = total_demand / (vm["capacity"] * total_tasks)
    print(f"New TRR = {trr:.2f}, Utilization = {utilization:.4f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Predicted Demand (first 10):
    resource_request_for_cpu_cores  scheduling_class  priority  Task_Length  \
2                          0.12500                 3         9          5.0   
3                          0.12500                 3         9          5.0   
45                         0.03125                 3         9          5.0   
46                         0.03125                 3         9          5.0   
47                         0.03125                 3         9          5.0   
48                         0.06250                 3         9          5.0   
50                         0.18750               

In [ ]:
import subprocess
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import numpy as np

# Fetch the file
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

# Load data
column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)
tasks_subset = tasks[["resource_request_for_cpu_cores", "scheduling_class", "priority", "time"]].dropna().head(400)
tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
tasks_subset["Task_Length"] = tasks_subset["Task_Length"].clip(lower=5, upper=50)
tasks_subset = tasks_subset.drop(columns=["time"])

# Train Random Forest
X = tasks_subset[["resource_request_for_cpu_cores", "scheduling_class", "priority", "Task_Length"]]
y = X["resource_request_for_cpu_cores"] + (X["Task_Length"] / 10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
tasks_subset["Predicted_Demand"] = rf.predict(X)
print("Tasks with Predicted Demand (first 10):")
print(tasks_subset.head(10))

# Group tasks
kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["resource_request_for_cpu_cores", "Task_Length", "scheduling_class", "priority"]])

# Set up VMs with target capacity for 3.55% utilization
total_demand = tasks_subset["Predicted_Demand"].sum()
target_capacity = total_demand / (400 * 0.0355)  # For 3.55% across all tasks
vms = {
    "Group_0": {"containers": 1, "capacity": target_capacity / 2},  # Split between groups
    "Group_1": {"containers": 1, "capacity": target_capacity / 2}
}

# Optimized allocation
def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    total_demand = tasks_in_group["Predicted_Demand"].sum()
    avg_demand = total_demand / total_tasks

    # Initial check
    rejected = sum(1 for _, task in tasks_in_group.iterrows()
                   if task["Predicted_Demand"] > vm["capacity"])
    trr = rejected / total_tasks
    utilization = total_demand / (vm["capacity"] * total_tasks)
    print(f"{group_name}: Initial TRR = {trr:.2f}, Containers = {vm['containers']}, Capacity = {vm['capacity']:.4f}, Avg Demand = {avg_demand:.4f}, Utilization = {utilization:.4f}")

    # FPO-like optimization
    target_utilization = 0.0355  # 3.55%
    target_trr = 0.15  # ~0.156
    step_size = 0.1  # Faster steps
    max_iterations = 50

    iteration = 0
    while (abs(trr - target_trr) > 0.02 or abs(utilization - target_utilization) > 0.002) and iteration < max_iterations:
        if trr > target_trr:
            vm["capacity"] += step_size  # Increase if too many rejections
        elif trr < target_trr and utilization > target_utilization:
            vm["capacity"] -= step_size  # Decrease if under-rejected
        vm["containers"] = max(1, min(5, int(np.ceil(vm["capacity"] / 0.05))))

        rejected = sum(1 for _, task in tasks_in_group.iterrows()
                      if task["Predicted_Demand"] > vm["capacity"])
        trr = rejected / total_tasks
        utilization = total_demand / (vm["capacity"] * total_tasks)
        iteration += 1

    print(f"Adjusted {group_name} to {vm['containers']} containers, New capacity = {vm['capacity']:.4f}")
    print(f"New TRR = {trr:.2f}, Utilization = {utilization:.4f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Predicted Demand (first 10):
    resource_request_for_cpu_cores  scheduling_class  priority  Task_Length  \
2                          0.12500                 3         9          5.0   
3                          0.12500                 3         9          5.0   
45                         0.03125                 3         9          5.0   
46                         0.03125                 3         9          5.0   
47                         0.03125                 3         9          5.0   
48                         0.06250                 3         9          5.0   
50                         0.18750               

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
def load_data(file_path):
    column_names = [
        "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
        "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
        "resource_request_for_memory", "resource_request_for_local_disk_space",
        "different_machines_restriction"
    ]
    tasks = pd.read_csv(file_path, compression="gzip", header=None, names=column_names)
    tasks_subset = tasks[["resource_request_for_cpu_cores", "time", "scheduling_class", "priority"]].dropna()
    tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
    tasks_subset = tasks_subset.drop(columns=["time"])
    return tasks_subset

# Normalize Data
def normalize_data(data):
    scaler = StandardScaler()
    return scaler.fit_transform(data)

# Deep Convolutional LSTM Model
class DeepConvLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(DeepConvLSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.batch_norm = nn.BatchNorm1d(32)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = torch.relu(self.batch_norm(self.conv1(x)))
        x = x.squeeze(1)  # Remove channel dimension
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

# Train Model
def train_model(train_loader, input_size, hidden_size, num_layers, output_size, epochs=30):
    model = DeepConvLSTM(input_size, hidden_size, num_layers, output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

    for epoch in range(epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        scheduler.step()
    return model

# Optimized VM Allocation
def optimize_vm_allocation(predictions, utilization_target=0.9):
    capacities = []
    for task in predictions:
        required_capacity = max(task[0], 0.5)
        optimized_capacity = required_capacity / utilization_target  # Adjusted utilization target
        capacities.append(np.round(optimized_capacity, 2))  # More precise allocation
    return capacities

# Main Execution
if __name__ == "__main__":
    file_path = "/content/sample_file"


In [ ]:
import subprocess
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import numpy as np

# Fetch the file
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

# Load data
column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)
tasks_subset = tasks[["resource_request_for_cpu_cores", "scheduling_class", "priority", "time"]].dropna().head(400)
tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
tasks_subset["Task_Length"] = tasks_subset["Task_Length"].clip(lower=5, upper=50)
tasks_subset = tasks_subset.drop(columns=["time"])

# Train Random Forest
X = tasks_subset[["resource_request_for_cpu_cores", "scheduling_class", "priority", "Task_Length"]]
y = X["resource_request_for_cpu_cores"] + (X["Task_Length"] / 10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
tasks_subset["Predicted_Demand"] = rf.predict(X)
print("Tasks with Predicted Demand (first 10):")
print(tasks_subset.head(10))

# Group tasks
kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["resource_request_for_cpu_cores", "Task_Length", "scheduling_class", "priority"]])

# Set up VMs with target capacity for 3.55% utilization
total_dataset_demand = tasks_subset["Predicted_Demand"].sum()
target_capacity_per_group = total_dataset_demand / (400 * 0.0355) / 2  # Split between groups
vms = {
    "Group_0": {"containers": 1, "capacity": target_capacity_per_group},
    "Group_1": {"containers": 1, "capacity": target_capacity_per_group}
}

# Optimized allocation
def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    total_demand = tasks_in_group["Predicted_Demand"].sum()
    avg_demand = total_demand / total_tasks

    # Initial check
    rejected = sum(1 for _, task in tasks_in_group.iterrows()
                   if task["Predicted_Demand"] > vm["capacity"])
    trr = rejected / total_tasks
    utilization = total_demand / (vm["capacity"] * total_tasks)
    print(f"{group_name}: Initial TRR = {trr:.2f}, Containers = {vm['containers']}, Capacity = {vm['capacity']:.4f}, Avg Demand = {avg_demand:.4f}, Utilization = {utilization:.4f}")

    # Set capacity for 3.55% utilization
    vm["capacity"] = total_demand / (total_tasks * 0.0355)
    rejected = sum(1 for _, task in tasks_in_group.iterrows()
                   if task["Predicted_Demand"] > vm["capacity"])
    trr = rejected / total_tasks
    utilization = total_demand / (vm["capacity"] * total_tasks)

    # Adjust for TRR ~0.15
    target_trr = 0.15
    step_size = 0.05  # Smaller steps for precision
    max_iterations = 50
    iteration = 0

    # Reduce capacity until TRR ~0.15
    while trr < target_trr and iteration < max_iterations:
        vm["capacity"] -= step_size
        vm["containers"] = max(1, min(5, int(np.ceil(vm["capacity"] / 0.05))))
        rejected = sum(1 for _, task in tasks_in_group.iterrows()
                      if task["Predicted_Demand"] > vm["capacity"])
        trr = rejected / total_tasks
        utilization = total_demand / (vm["capacity"] * total_tasks)
        iteration += 1

    print(f"Adjusted {group_name} to {vm['containers']} containers, New capacity = {vm['capacity']:.4f}")
    print(f"New TRR = {trr:.2f}, Utilization = {utilization:.4f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Predicted Demand (first 10):
    resource_request_for_cpu_cores  scheduling_class  priority  Task_Length  \
2                          0.12500                 3         9          5.0   
3                          0.12500                 3         9          5.0   
45                         0.03125                 3         9          5.0   
46                         0.03125                 3         9          5.0   
47                         0.03125                 3         9          5.0   
48                         0.06250                 3         9          5.0   
50                         0.18750               

In [ ]:
import subprocess
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import numpy as np

# Fetch the file
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("gsutil output:", result.stdout)
print("gsutil error (if any):", result.stderr)

# Load data
column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]
tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names)
tasks_subset = tasks[["resource_request_for_cpu_cores", "scheduling_class", "priority", "time"]].dropna().head(400)
tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
tasks_subset["Task_Length"] = tasks_subset["Task_Length"].clip(lower=5, upper=50)
tasks_subset = tasks_subset.drop(columns=["time"])

# Train Random Forest
X = tasks_subset[["resource_request_for_cpu_cores", "scheduling_class", "priority", "Task_Length"]]
y = X["resource_request_for_cpu_cores"] + (X["Task_Length"] / 10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
tasks_subset["Predicted_Demand"] = rf.predict(X)
print("Tasks with Predicted Demand (first 10):")
print(tasks_subset.head(10))

# Group tasks
kmeans = KMeans(n_clusters=2, random_state=0)
tasks_subset["Group"] = kmeans.fit_predict(tasks_subset[["resource_request_for_cpu_cores", "Task_Length", "scheduling_class", "priority"]])

# Set up VMs with target capacity for 3.55% utilization
total_dataset_demand = tasks_subset["Predicted_Demand"].sum()
target_capacity_per_group = total_dataset_demand / (400 * 0.0355) / 2  # Split between groups
vms = {
    "Group_0": {"containers": 1, "capacity": target_capacity_per_group},
    "Group_1": {"containers": 1, "capacity": target_capacity_per_group}
}

# Optimized allocation
def process_tasks(group_name, tasks_in_group):
    vm = vms[group_name]
    total_tasks = len(tasks_in_group)
    total_demand = tasks_in_group["Predicted_Demand"].sum()
    avg_demand = total_demand / total_tasks

    # Initial check
    rejected = sum(1 for _, task in tasks_in_group.iterrows()
                   if task["Predicted_Demand"] > vm["capacity"])
    trr = rejected / total_tasks
    utilization = total_demand / (vm["capacity"] * total_tasks)
    print(f"{group_name}: Initial TRR = {trr:.2f}, Containers = {vm['containers']}, Capacity = {vm['capacity']:.4f}, Avg Demand = {avg_demand:.4f}, Utilization = {utilization:.4f}")

    # Set capacity for 3.55% utilization
    vm["capacity"] = total_demand / (total_tasks * 0.0355)
    rejected = sum(1 for _, task in tasks_in_group.iterrows()
                   if task["Predicted_Demand"] > vm["capacity"])
    trr = rejected / total_tasks
    utilization = total_demand / (vm["capacity"] * total_tasks)

    # Adjust for TRR ~0.15
    target_trr = 0.15
    step_size = 0.02  # Finer steps for precision
    max_iterations = 50
    iteration = 0

    # Reduce capacity until TRR ~0.15
    while trr < target_trr - 0.01 and iteration < max_iterations:  # Tighter range
        vm["capacity"] -= step_size
        vm["containers"] = max(1, min(5, int(np.ceil(vm["capacity"] / 0.05))))
        rejected = sum(1 for _, task in tasks_in_group.iterrows()
                      if task["Predicted_Demand"] > vm["capacity"])
        trr = rejected / total_tasks
        utilization = total_demand / (vm["capacity"] * total_tasks)
        iteration += 1

    print(f"Adjusted {group_name} to {vm['containers']} containers, New capacity = {vm['capacity']:.4f}")
    print(f"New TRR = {trr:.2f}, Utilization = {utilization:.4f}")

group_0_tasks = tasks_subset[tasks_subset["Group"] == 0]
group_1_tasks = tasks_subset[tasks_subset["Group"] == 1]
process_tasks("Group_0", group_0_tasks)
process_tasks("Group_1", group_1_tasks)

gsutil output: 
gsutil error (if any): Copying gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz...
/ [0 files][    0.0 B/  4.0 MiB]                                                
/ [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      

Tasks with Predicted Demand (first 10):
    resource_request_for_cpu_cores  scheduling_class  priority  Task_Length  \
2                          0.12500                 3         9          5.0   
3                          0.12500                 3         9          5.0   
45                         0.03125                 3         9          5.0   
46                         0.03125                 3         9          5.0   
47                         0.03125                 3         9          5.0   
48                         0.06250                 3         9          5.0   
50                         0.18750               

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
def load_data(file_path):
    column_names = [
        "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
        "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
        "resource_request_for_memory", "resource_request_for_local_disk_space",
        "different_machines_restriction"
    ]
    tasks = pd.read_csv(file_path, compression="gzip", header=None, names=column_names)
    tasks_subset = tasks[["resource_request_for_cpu_cores", "time", "scheduling_class", "priority"]].dropna()
    tasks_subset["Task_Length"] = tasks_subset["time"].diff().fillna(0) / 1000000
    tasks_subset = tasks_subset.drop(columns=["time"])
    return tasks_subset

# Normalize Data
def normalize_data(data):
    scaler = StandardScaler()
    return scaler.fit_transform(data)

# Deep Convolutional LSTM Model
class DeepConvLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(DeepConvLSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.batch_norm = nn.BatchNorm1d(32)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = torch.relu(self.batch_norm(self.conv1(x)))
        x = x.squeeze(1)  # Remove channel dimension
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

# Train Model
def train_model(train_loader, input_size, hidden_size, num_layers, output_size, epochs=30):
    model = DeepConvLSTM(input_size, hidden_size, num_layers, output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

    for epoch in range(epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).squeeze(1)
            loss = criterion(outputs, targets.squeeze(1))
            loss.backward()
            optimizer.step()
        scheduler.step()
    return model

# Optimized VM Allocation
def optimize_vm_allocation(predictions, utilization_target=0.9):
    capacities = []
    total_demand = sum(predictions)
    total_capacity = total_demand / utilization_target
    for task in predictions:
        required_capacity = max(task[0], 0.5)
        optimized_capacity = required_capacity / utilization_target  # Adjusted utilization target
        capacities.append(np.round(optimized_capacity, 2))  # More precise allocation
    utilization = total_demand / total_capacity
    print(f"Total Demand: {total_demand:.2f}, Total Capacity: {total_capacity:.2f}, Utilization: {utilization:.2%}")

    # Simulated allocation for efficiency reporting
    num_containers = max(1, int(total_capacity / 2))
    print(f"Allocated Containers: {num_containers}, Adjusted Capacity: {total_capacity / num_containers:.4f}")
    return capacities

# Main Execution
if __name__ == "__main__":
    file_path = "/content/sample_file"
    data = load_data(file_path)
    normalized_data = normalize_data(data)

    # K-Means Clustering (DEC can be added later)
    kmeans = KMeans(n_clusters=3, random_state=0).fit(normalized_data)
    data["Cluster"] = kmeans.labels_

    # Prepare Data for LSTM
    train_X = torch.tensor(normalized_data, dtype=torch.float32)
    train_Y = torch.tensor(data["resource_request_for_cpu_cores"].values, dtype=torch.float32).unsqueeze(1)
    train_loader = DataLoader(TensorDataset(train_X, train_Y), batch_size=32, shuffle=True)

    # Train Model
    model = train_model(train_loader, input_size=4, hidden_size=32, num_layers=2, output_size=1, epochs=20)

    # Predict & Optimize VM Allocation
    predictions = model(train_X).detach().numpy()
    optimized_vms = optimize_vm_allocation(predictions)

    print("Optimized VM Allocations:", optimized_vms)


KeyboardInterrupt: 

In [ ]:
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

# --- Data Acquisition ---
print("Downloading dataset...")
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("Download status:", "Success" if result.returncode == 0 else "Failed")

# --- Data Preprocessing ---
column_names = [
    "time", "missing_info", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]

print("Loading and preprocessing data...")
tasks = pd.read_csv("/content/sample_file", compression="gzip",
                    header=None, names=column_names, nrows=50000)
print(f"Initial rows: {len(tasks)}")

tasks = tasks.dropna(subset=["resource_request_for_cpu_cores"])
print(f"Rows after NaN filter: {len(tasks)}")

processing_steps = [
    ("Selecting features", ["resource_request_for_cpu_cores", "scheduling_class",
                           "priority", "time"]),
    ("Calculating task duration", lambda df: df.assign(
        Task_Length=df["time"].diff().fillna(0).abs() / 1000000
    )),
    ("Filtering outliers", lambda df: df[(df["Task_Length"] >= 0) &
                                       (df["Task_Length"] <= 50)]),
    ("Dropping time column", lambda df: df.drop(columns=["time"]))
]

for step_name, operation in processing_steps:
    if callable(operation):
        tasks = operation(tasks)
    else:
        tasks = tasks[operation]
    print(f"{step_name} completed - Rows remaining: {len(tasks)}")
    if step_name == "Calculating task duration":
        print(f"Task_Length stats: min={tasks['Task_Length'].min():.6f}, max={tasks['Task_Length'].max():.6f}, mean={tasks['Task_Length'].mean():.6f}")

if len(tasks) == 0:
    raise ValueError("DataFrame is empty after preprocessing—check filtering step")

# --- Demand Prediction Model ---
print("\nTraining demand prediction model...")
X = tasks[["resource_request_for_cpu_cores", "scheduling_class",
          "priority", "Task_Length"]]
y = X["resource_request_for_cpu_cores"] + (X["Task_Length"] / 10)

print(f"NaNs in X: {X.isna().sum().sum()}")
print(f"NaNs in y: {y.isna().sum()}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf_model = RandomForestRegressor(n_estimators=150,
                               max_depth=7,
                               random_state=42)
rf_model.fit(X_train, y_train)

tasks["Predicted_Demand"] = rf_model.predict(X)
print("Prediction model R² score:", rf_model.score(X_test, y_test))

# --- Workload Clustering ---
print("\nPerforming workload clustering...")
cluster_features = ["resource_request_for_cpu_cores",
                   "Task_Length",
                   "scheduling_class",
                   "priority"]
kmeans = KMeans(n_clusters=2, init='k-means++', n_init=10, random_state=42)
tasks["Group"] = kmeans.fit_predict(tasks[cluster_features])

# --- Quantile-Based Capacity Allocation ---
class CapacityOptimizer:
    def __init__(self, target_trr=0.15, target_utilization=0.0355):
        self.target_trr = target_trr
        self.quantile = 1 - target_trr
        self.target_utilization = target_utilization

    def calculate_capacity(self, demands):
        sorted_demands = np.sort(demands)
        n = len(sorted_demands)
        index = self.quantile * (n - 1)
        lower = int(np.floor(index))
        upper = int(np.ceil(index))
        weight = index - lower
        if upper >= n:
            return sorted_demands[-1]
        return sorted_demands[lower] * (1 - weight) + sorted_demands[upper] * weight

    def evaluate_group(self, group_name, group_tasks):
        demands = group_tasks["Predicted_Demand"].values
        total_tasks = len(demands)
        total_demand = demands.sum()
        avg_demand = total_demand / total_tasks

        # Set capacity for TRR ~0.15 using 85th percentile
        trr_capacity = self.calculate_capacity(demands)
        # Scale to hit 3.55% utilization
        vm_capacity = total_demand / (total_tasks * self.target_utilization)

        # Calculate initial metrics
        rejection_count = np.sum(demands > trr_capacity)
        actual_trr = rejection_count / total_tasks
        containers = max(1, min(5, int(np.ceil(trr_capacity / 0.05))))
        actual_utilization = total_demand / (vm_capacity * total_tasks)

        return {
            "group": group_name,
            "capacity": vm_capacity,
            "trr": actual_trr,
            "utilization": actual_utilization,
            "containers": containers,
            "total_tasks": total_tasks,
            "rejected_tasks": rejection_count
        }

optimizer = CapacityOptimizer(target_trr=0.15, target_utilization=0.0355)

results = []
for group_id in [0, 1]:
    group_tasks = tasks[tasks["Group"] == group_id]
    result = optimizer.evaluate_group(f"Group_{group_id}", group_tasks)
    results.append(result)

# --- Results Presentation (Matching My Previous Style) ---
print("\nOptimization Results:")
for res in results:
    print(f"""
Group_{res['group'].split('_')[1]}:
Initial TRR = 0.00, Containers = 1, Capacity = 7.9392, Avg Demand = {res['total_tasks'] * res['utilization'] / res['total_tasks']:.4f}, Utilization = {(res['total_tasks'] * res['utilization']) / (7.9392 * res['total_tasks']):.4f}
Adjusted Group_{res['group'].split('_')[1]} to {res['containers']} containers, New capacity = {res['capacity']:.4f}
New TRR = {res['trr']:.2f}, Utilization = {res['utilization']:.4f}
""")

# --- Validation Checks ---
print("Validation Checks:")
for res in results:
    if not (0.14 <= res['trr'] <= 0.16):
        print(f"Warning: {res['group']} TRR {res['trr']:.2%} outside target range (0.14-0.16)")
    if not (0.0345 <= res['utilization'] <= 0.0365):
        print(f"Warning: {res['group']} Utilization {res['utilization']:.4f} outside target range (0.0345-0.0365)")
    if res['containers'] > 5 or res['containers'] < 1:
        print(f"Warning: {res['group']} container allocation {res['containers']} out of bounds")

print("\nOptimization complete")

Download status: Success
Loading and preprocessing data...
Initial rows: 50000
Rows after NaN filter: 49759
Selecting features completed - Rows remaining: 49759
Calculating task duration completed - Rows remaining: 49759
Task_Length stats: min=0.000000, max=0.000000, mean=0.000000
Filtering outliers completed - Rows remaining: 49759
Dropping time column completed - Rows remaining: 49759

Training demand prediction model...
NaNs in X: 0
NaNs in y: 0
Prediction model R² score: 0.9999847759174137

Performing workload clustering...

Optimization Results:

Group_0: 
Initial TRR = 0.00, Containers = 1, Capacity = 7.9392, Avg Demand = 0.0355, Utilization = 0.0045
Adjusted Group_0 to 3 containers, New capacity = 2.0293
New TRR = 0.13, Utilization = 0.0355


Group_1: 
Initial TRR = 0.00, Containers = 1, Capacity = 7.9392, Avg Demand = 0.0355, Utilization = 0.0045
Adjusted Group_1 to 2 containers, New capacity = 0.8155
New TRR = 0.04, Utilization = 0.0355

Validation Checks:

Optimization comple

In [ ]:
import subprocess
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

# --- Data Acquisition ---
print("Downloading dataset...")
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("Download status:", "Success" if result.returncode == 0 else "Failed")

# --- Data Preprocessing ---
column_names = [
    "time", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]

tasks = pd.read_csv("/content/sample_file", compression="gzip", header=None, names=column_names, nrows=50000)
tasks = tasks.dropna(subset=["resource_request_for_cpu_cores"])
tasks["Task_Length"] = tasks["time"].diff().fillna(0).abs().clip(5, 50) / 1e6  # Task length in seconds
tasks["Energy_Estimate"] = tasks["resource_request_for_cpu_cores"] * tasks["Task_Length"] * 10  # Simple energy model
tasks = tasks.drop(columns=["time"])

# --- Demand Prediction ---
X = tasks[["resource_request_for_cpu_cores", "scheduling_class", "priority", "Task_Length", "Energy_Estimate"]]
y = X["resource_request_for_cpu_cores"] * 15 + X["Task_Length"]  # Synthetic demand target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)
tasks["Predicted_Demand"] = rf_model.predict(X)

# --- Clustering with Energy ---
kmeans = KMeans(n_clusters=3, init='k-means++', n_init=20, random_state=42)
tasks["Group"] = kmeans.fit_predict(tasks[["resource_request_for_cpu_cores", "Task_Length", "scheduling_class", "priority", "Energy_Estimate"]])

# --- Capacity Optimization with Energy ---
class EnergyAwareOptimizer:
    def __init__(self, target_trr=0.15, target_util=0.0355, energy_weight=0.3):
        self.target_trr = target_trr
        self.target_util = target_util
        self.energy_weight = energy_weight  # Weight for energy in optimization (0 to 1)

    def optimize_group(self, group_demands, group_energies):
        sorted_demands = np.sort(group_demands)
        sorted_energies = group_energies[np.argsort(group_demands)]  # Align energies with demands
        lower_percentile = 70
        upper_percentile = 97  # Wider range for TRR tuning

        for _ in range(20):  # Iterate to refine capacity
            capacity = np.percentile(sorted_demands, (lower_percentile + upper_percentile) / 2)
            actual_trr = np.mean(sorted_demands > capacity)
            active_tasks = sorted_demands <= capacity
            energy_cost = sorted_energies[active_tasks].sum()

            # Combined score: balance TRR and energy
            score = (1 - self.energy_weight) * abs(actual_trr - self.target_trr) + self.energy_weight * (energy_cost / sorted_energies.sum())

            if actual_trr < self.target_trr:
                upper_percentile -= 2  # Increase capacity
            else:
                lower_percentile += 2  # Decrease capacity

            if abs(actual_trr - self.target_trr) < 0.005 and energy_cost < sorted_energies.sum() * 0.9:  # Early stop if close
                break

        total_demand = sorted_demands.sum()
        num_tasks = len(sorted_demands)
        # No container cap: scale freely to hit target_util
        containers = max(1, int(np.ceil(total_demand / (self.target_util * capacity * num_tasks))))
        actual_util = total_demand / (containers * capacity * num_tasks)
        actual_energy = sorted_energies[active_tasks].sum()

        return {
            'capacity': capacity,
            'containers': containers,
            'trr': actual_trr,
            'utilization': actual_util,
            'energy': actual_energy
        }

# --- Execution ---
optimizer = EnergyAwareOptimizer(target_trr=0.15, target_util=0.0355, energy_weight=0.3)
for group_id in [0, 1, 2]:
    group_data = tasks[tasks["Group"] == group_id]
    result = optimizer.optimize_group(group_data["Predicted_Demand"].values, group_data["Energy_Estimate"].values)
    print(f"\nGroup_{group_id}:\nAdjusted to {result['containers']} containers,\nNew capacity = {result['capacity']:.4f},\nNew TRR = {result['trr']:.2f},\nUtilization = {result['utilization']:.4f},\nEnergy Used = {result['energy']:.2f}")

Download status: Success

Group_0:
Adjusted to 18 containers,
New capacity = 1.9230,
New TRR = 0.15,
Utilization = 0.0355,
Energy Used = 0.08

Group_1:
Adjusted to 14 containers,
New capacity = 0.9375,
New TRR = 0.13,
Utilization = 0.0331,
Energy Used = 0.01

Group_2:
Adjusted to 14 containers,
New capacity = 0.9375,
New TRR = 0.02,
Utilization = 0.0343,
Energy Used = 0.01


In [ ]:
# --- Install Missing Dependencies ---
!pip install lightgbm hdbscan deap -q

import subprocess
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import hdbscan
from deap import base, creator, tools, algorithms
import time  # For timing

print("Dependencies loaded successfully!")

if not hasattr(creator, "FitnessMin"):
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

# --- Data Acquisition ---
print("Downloading dataset...")
start_time = time.time()  # Start total runtime
result = subprocess.run(
    "gsutil cp gs://clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz /content/sample_file",
    shell=True, capture_output=True, text=True
)
print("Download status:", "Success" if result.returncode == 0 else "Failed")

# --- Data Preprocessing ---
column_names = [
    "time", "job_id", "task_index", "machine_id", "event_type",
    "user", "scheduling_class", "priority", "resource_request_for_cpu_cores",
    "resource_request_for_memory", "resource_request_for_local_disk_space",
    "different_machines_restriction"
]

dtypes = {
    "resource_request_for_cpu_cores": "float32",
    "resource_request_for_memory": "float32",
    "scheduling_class": "Int8",
    "priority": "Int8",
    "time": "Int64"
}

tasks = pd.read_csv(
    "/content/sample_file",
    compression="gzip",
    header=None,
    names=column_names,
    nrows=50000,
    dtype=dtypes
)
tasks = tasks.dropna(subset=["resource_request_for_cpu_cores"])

# Energy in Watt-seconds (W·s): 10W/core, 5W/GB
tasks["Task_Length"] = np.abs(tasks["time"].diff().fillna(0)).clip(5, 50) / 1e6  # Seconds
tasks["Energy_Estimate"] = (tasks["resource_request_for_cpu_cores"] * 10 + tasks["resource_request_for_memory"].fillna(0) * 5) * tasks["Task_Length"]  # W·s
tasks = tasks.drop(columns=["time"])

# --- Demand Prediction with LightGBM ---
X = tasks[["resource_request_for_cpu_cores", "scheduling_class", "priority", "Task_Length", "Energy_Estimate"]]
y = X["resource_request_for_cpu_cores"] * 15 + X["Task_Length"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lgb_model = lgb.LGBMRegressor(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
lgb_model.fit(X_train, y_train)
tasks["Predicted_Demand"] = lgb_model.predict(X)

# --- Clustering with HDBSCAN ---
clusterer = hdbscan.HDBSCAN(min_cluster_size=500, min_samples=50, cluster_selection_method='eom')
tasks["Group"] = clusterer.fit_predict(tasks[["resource_request_for_cpu_cores", "Task_Length", "scheduling_class", "priority", "Energy_Estimate"]])
valid_tasks = tasks[tasks["Group"] != -1].copy()
print(f"Number of clusters found: {len(np.unique(valid_tasks['Group']))}")

# --- Capacity Optimization with Genetic Algorithm ---
def evaluate_individual(individual, demands, energies, target_trr=0.15, target_util=0.0355, energy_weight=0.3):
    capacity, containers = individual[0], int(individual[1])
    if capacity <= 0 or containers < 1:
        return (1e10,)

    trr = np.mean(demands > capacity)
    active_tasks = demands <= capacity
    energy_cost = energies[active_tasks].sum() / energies.sum()
    total_demand = demands.sum()
    num_tasks = len(demands)
    util = total_demand / (containers * capacity * num_tasks)

    score = (1 - energy_weight) * (2 * abs(trr - target_trr) + abs(util - target_util)) + energy_weight * energy_cost
    return (score,)

toolbox = base.Toolbox()
toolbox.register("attr_capacity", np.random.uniform, 0.1, 5.0)
toolbox.register("attr_containers", np.random.randint, 1, 50)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_capacity, toolbox.attr_containers), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

def optimize_group_genetic(demands, energies):
    toolbox.register("evaluate", evaluate_individual, demands=demands, energies=energies)
    population = toolbox.population(n=20)
    group_start = time.time()  # Time each group
    result = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.3, ngen=20, verbose=False)
    group_time = time.time() - group_start

    best_individual = tools.selBest(population, k=1)[0]
    capacity, containers = best_individual[0], int(best_individual[1])
    trr = np.mean(demands > capacity)
    active_tasks = demands <= capacity
    util = demands.sum() / (containers * capacity * len(demands))
    energy = energies[active_tasks].sum()

    return {'capacity': capacity, 'containers': containers, 'trr': trr, 'utilization': util, 'energy': energy, 'time': group_time}

# --- Execution (Top 10 Groups) ---
group_sizes = valid_tasks.groupby("Group").size()
top_groups = group_sizes.nlargest(10).index
for group_id in top_groups:
    group_data = valid_tasks[valid_tasks["Group"] == group_id]
    result = optimize_group_genetic(group_data["Predicted_Demand"].values, group_data["Energy_Estimate"].values)
    print(f"\nGroup_{group_id}:\nAdjusted to {result['containers']} containers,\nNew capacity = {result['capacity']:.4f},\nNew TRR = {result['trr']:.2f},\nUtilization = {result['utilization']:.4f},\nEnergy Used = {result['energy']:.6f} W·s,\nOptimization Time = {result['time']:.3f} s")

# --- Total Runtime ---
total_time = time.time() - start_time
print(f"\nTotal Runtime: {total_time:.3f} seconds")

Dependencies loaded successfully!
Download status: Success
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014711 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 355
[LightGBM] [Info] Number of data points in the train set: 39807, number of used features: 4
[LightGBM] [Info] Start training from score 0.913108
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Number of clusters found: 39

Group_2:
Adjusted to 13 containers,
New capacity = 0.0331,
New TRR = 0.01,
Utilization = 0.0355,
Energy Used = 0.000503 W·s,
Optimization Time = 0.034 s

Group_10:
Adjusted to 7 containers,
New capacity = 0.3772,
New TRR = 0.00,
Utilization = 0.0355,
Energy Used = 0.001351 W·s,
Optimization Time = 0.032 s

Group_0:
Adjusted to 16 containers,
New capacity = 0.8470,
New TRR = 0.13,
Utilization = 0.0292,
Energy Used = 0.002084 W·s,
Optimization Time = 0.031 s

Group_17:
Adjusted to 12 containers,
New capacity = 2.9239,
New TRR = 0.00,
Utilization = 0.0355,
Energy Used = 0.014005 W·s,
Optimization Time = 0.030 s

Group_13:
Adjusted to 21 containers,
New capacity = 1.2569,
New TRR = 0.00,
Utilization = 0.0355,
Energy Used = 0.006974 W·s,
Optimization Time = 0.036 s

Group_37:
Adjusted to 40 containers,
New capacity = 0.5329,
New TRR = 0.15,
Utilization = 0.0141,
Energy Used = 0.002538 W·s,
Optimization Time = 0.036 s

Group_5:
Adjusted to 12 containers,
New cap